In [1]:
from trdg.generators import GeneratorFromStrings
import cv2
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa
import os

2024-06-04 14:13:21.403880: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 14:13:21.628250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 14:13:21.628302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 14:13:21.629457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 14:13:21.725315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 14:13:21.727689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
def thresholding(img):
    """
    Applies thresholding to the input image
    """
    img.save("temp.jpg")
    img = cv2.imread("temp.jpg", cv2.IMREAD_GRAYSCALE)

    _, im_gray_th_otsu = cv2.threshold(img, 128, 192, cv2.THRESH_OTSU)

    _, im_gray_th = cv2.threshold(im_gray_th_otsu, 128, 255, cv2.THRESH_BINARY)

    im_gray_th = cv2.bitwise_not(im_gray_th)

    return Image.fromarray(im_gray_th)

In [8]:
# Create a generator object 
generator = GeneratorFromStrings(
    ['B', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'N', 'P', 'R', 'S', 'T', 'V', 'W', 'X', 'Z', '1', '2','3','4','5','6','7','8','9','0'],
    count=5,
    fonts=['../lp_template_files/font_dutch_lp.ttf'],
    skewing_angle=7,
    random_skew=True,
    fit=True,
    margins=(0,0,0,0),
    distorsion_type=1,
    background_type=1,
    text_color='#000000',
    size=40,
    blur=2,
    random_blur=True,
)


img_count = 0

for img, lbl in generator:
    # apply thresholding
    img = thresholding(img)

    # find the bounding box of the image
    cnts, _ = cv2.findContours(np.array(img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundingBox = [cv2.boundingRect(c) for c in cnts][0]

    # crop the image to the bounding box
    img = img.crop((boundingBox[0], boundingBox[1], boundingBox[0]+boundingBox[2], boundingBox[1]+boundingBox[3]))

    # resize the image to 20x30
    img = img.resize((20, 30))

    # create blank black image
    new_img = Image.new("RGB", (23, 33), "black")

    # paste the original image on the blank image in middle
    width, height = img.size
    x = 23//2 - width//2
    y = 33//2 - height//2
    new_img.paste(img, (x, y))

    # save the image
    new_img.save(f'../../images/train_ocr_test1/{lbl}-{img_count}.png')
    img_count += 1
    

In [29]:
image_dir = "../../images/train_ocr_test1"
print(len(os.listdir(image_dir)))

image_array = []
names = []
for img in os.listdir(image_dir):
    image = Image.open(os.path.join(image_dir, img))
    image_array.append(np.array(image))
    names.append(img)

10


In [31]:
aug = iaa.Sequential([
    iaa.MotionBlur(k=(3, 5)),
])

for i in range(len(image_array)):
    image = Image.fromarray(image_array[i])
    image = aug(image=np.array(image))
    image = Image.fromarray(image)
    # convert all non-black pixels to white
    image = image.point(lambda p: p > 128 and 255)
    image.save(f'../../images/train_ocr_test/{names[i]}')